# DenseNet121_on_CheXpert
[05_Optimizing_AUROC_Loss_with_DenseNet121_on_CheXpert.ipynb](https://github.com/Optimization-AI/LibAUC/blob/main/examples/05_Optimizing_AUROC_Loss_with_DenseNet121_on_CheXpert.ipynb)


---

## < Summary >
### 1. Pre-training
- train dataset : 191,027 images
- test dataset : 202 images
- 5 categories : Cardiomegaly, Edema, Consolidation, Atelectasis, Pleural Effusion
- pre-trained model : DenseNet121
- epoch 2
- Loss : CrossEntropyLoss
- Optimizer : Adam(lr=1e-4)
- BATCH_SIZE : 32
- Best Val_AUC : 

### 2. Optimizing AUCM Loss
- train dataset : 227,395 images
- test dataset : 202 images
- choose one categories
- pre-trained model : DenseNet121 or saved model at first step
- epoch 2
- Loss : AUCMLoss
- optimizer : PESG(lr=0.05)
- BATCH_SIZE : 32
- Best Val_AUC : 

### 3. Multi-Label Training
- train dataset : 191,027 images
- test dataset : 202 images
- 5 categories : Cardiomegaly, Edema, Consolidation, Atelectasis, Pleural Effusion
- pre-trained model : DenseNet121
- epoch 2
- Loss : AUCM_MultiLabel
- optimizer : PESG(lr=0.1)
- BATCH_SIZE : 32
- Best Val_AUC : 





---

## Importing LibAUC

In [1]:
from libauc.losses import AUCMLoss, CrossEntropyLoss, AUCM_MultiLabel
from libauc.optimizers import PESG, Adam
from libauc.models import DenseNet121, DenseNet169, DenseNet201, DenseNet161
from libauc.datasets import CheXpert

import torch 
from PIL import Image
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from sklearn.metrics import roc_auc_score

from torch.utils.tensorboard import SummaryWriter


## Reproducibility

In [2]:
def set_all_seeds(SEED):
    # REPRODUCIBILITY
    torch.manual_seed(SEED)
    np.random.seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

---
## 1. Pretraining
- Multi-label classification (5 tasks)
- Adam + CrossEntropy Loss
- This step is optional

In [3]:
# dataloader
root = 'D:/Data/AI604_project/CheXpert-v1.0-small/CheXpert-v1.0-small/'
# Index: -1 denotes multi-label mode including 5 diseases
traindSet = CheXpert(csv_path=root+'train.csv', image_root_path=root, use_upsampling=False, use_frontal=True, image_size=224, mode='train', class_index=-1)
testSet =  CheXpert(csv_path=root+'valid.csv',  image_root_path=root, use_upsampling=False, use_frontal=True, image_size=224, mode='valid', class_index=-1)
trainloader =  torch.utils.data.DataLoader(traindSet, batch_size=32, num_workers=2, shuffle=True)
testloader =  torch.utils.data.DataLoader(testSet, batch_size=32, num_workers=2, shuffle=False)

C:\Users\Hyunmin\Anaconda3\envs\pytorch10\lib\site-packages\libauc\datasets\chexpert.py:35: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df['Path'] = self.df['Path'].str.replace('CheXpert-v1.0-small/', '')
C:\Users\Hyunmin\Anaconda3\envs\pytorch10\lib\site-packages\libauc\datasets\chexpert.py:36: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df['Path'] = self.df['Path'].str.replace('CheXpert-v1.0/', '')


Multi-label mode: True, Number of classes: [5]
------------------------------
Found 191027 images in total, 23385 positive images, 167642 negative images
Cardiomegaly(C0): imbalance ratio is 0.1224

Found 191027 images in total, 61493 positive images, 129534 negative images
Edema(C1): imbalance ratio is 0.3219

Found 191027 images in total, 12983 positive images, 178044 negative images
Consolidation(C2): imbalance ratio is 0.0680

Found 191027 images in total, 59583 positive images, 131444 negative images
Atelectasis(C3): imbalance ratio is 0.3119

Found 191027 images in total, 76899 positive images, 114128 negative images
Pleural Effusion(C4): imbalance ratio is 0.4026

Multi-label mode: True, Number of classes: [5]
------------------------------
Found 202 images in total, 66 positive images, 136 negative images
Cardiomegaly(C0): imbalance ratio is 0.3267

Found 202 images in total, 42 positive images, 160 negative images
Edema(C1): imbalance ratio is 0.2079

Found 202 images in total

In [4]:
print(len(trainloader))
print(len(testloader))

5970
7


In [5]:
# paramaters
SEED = 123
BATCH_SIZE = 32
lr = 1e-4
weight_decay = 1e-5

name = 'DenseNet121' # ['DenseNet121', 'DenseNet169', 'DenseNet201', 'DenseNet161']

if name == 'DenseNet121' :
    model_name = DenseNet121
elif name == 'DenseNet169' :
    model_name = DenseNet169
elif name == 'DenseNet201' :
    model_name = DenseNet201
else :
    model_name = DenseNet161

writer = SummaryWriter('CheXpert_tensorboard')

# model
set_all_seeds(SEED)
model = model_name(pretrained=True, last_activation=None, activations='relu', num_classes=5)   
model = model.cuda()
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
    (elu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
        (elu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
        (elu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.01, affine=True, track_running_stats=True)
        (elu

In [6]:
# define loss & optimizer
CELoss = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [7]:
best_val_auc = 0 
loss_total = []

for epoch in range(2):
    for idx, data in enumerate(trainloader):
      train_data, train_labels = data
      # print(train_data.shape)    # torch.Size([32,3,224,224])
      # print(train_labels)        # torch.Size([32, 5]) → exapmle :: [[0., 1., 0., 0., 0.], [0., 0., 0., 1., 1.], [0., 1., 0., 0., 1.], 
      # print(train_labels.shape)
      train_data, train_labels  = train_data.cuda(), train_labels.cuda()
      y_pred = model(train_data)
      # print("y_pred shape", y_pred.shape) # torch.Size([32, 5])
      # print(y_pred) # [ 1.9280e-01,  1.2542e-04, -1.2416e+00,  4.4322e-01,  6.8606e-01], [-2.8613e-03, -5.7308e-01, -8.1134e-01,  6.5988e-01,  5.8159e-01], [ 5.5615e-01, -2.8245e-01, -8.8850e-01,  1.6961e-01,  4.4384e-01],
      loss = CELoss(y_pred, train_labels)

      loss_total += [loss.item()]

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # validation  
      if idx % 400 == 0:
         model.eval()
         with torch.no_grad():    
              test_pred = []
              test_true = [] 
              for jdx, data in enumerate(testloader):
                  test_data, test_labels = data
                  test_data = test_data.cuda()
                  y_pred = model(test_data)
                  test_pred.append(y_pred.cpu().detach().numpy())
                  test_true.append(test_labels.numpy())
            
              test_true = np.concatenate(test_true)
              test_pred = np.concatenate(test_pred)
              val_auc_mean =  roc_auc_score(test_true, test_pred) 

              writer.add_scalar('Pretraining/CELoss', np.mean(loss_total), epoch*len(trainloader) + idx)
              writer.add_scalar('Pretraining/VAL AUC', np.mean(val_auc_mean), epoch*len(trainloader) + idx)
              model.train()
              
              if best_val_auc < val_auc_mean:
                 best_val_auc = val_auc_mean
                 torch.save(model.state_dict(), 'DenseNet121_pretrained_model.pth')

              print ('Epoch=%s, BatchID=[%s/%s] | Loss=%f, Val_AUC=%.4f, Best_Val_AUC=%.4f'%(epoch, idx, len(trainloader), loss.item(), val_auc_mean, best_val_auc ))

print ('Best Val_AUC is %.4f' % best_val_auc)

Epoch=0, BatchID=[0/5970] | Loss=0.717473, Val_AUC=0.5088, Best_Val_AUC=0.5088
Epoch=0, BatchID=[400/5970] | Loss=0.405873, Val_AUC=0.8421, Best_Val_AUC=0.8421
Epoch=0, BatchID=[800/5970] | Loss=0.349457, Val_AUC=0.8619, Best_Val_AUC=0.8619
Epoch=0, BatchID=[1200/5970] | Loss=0.370392, Val_AUC=0.8580, Best_Val_AUC=0.8619


KeyboardInterrupt: 

---
## 2. Optimizing AUCM Loss
- Binary Classification
- PESG + AUCM Loss

In [ ]:
# parameters
class_id = 1 # 0:Cardiomegaly, 1:Edema, 2:Consolidation, 3:Atelectasis, 4:Pleural Effusion 

# You can set use_upsampling=True and pass the class name by upsampling_cols=['Cardiomegaly'] to do upsampling. This may improve the performance
traindSet = CheXpert(csv_path=root+'train.csv', image_root_path=root, use_upsampling=True, use_frontal=True, image_size=224, mode='train', class_index=class_id)
testSet =  CheXpert(csv_path=root+'valid.csv',  image_root_path=root, use_upsampling=False, use_frontal=True, image_size=224, mode='valid', class_index=class_id)
trainloader =  torch.utils.data.DataLoader(traindSet, batch_size=32, num_workers=2, shuffle=True)
testloader =  torch.utils.data.DataLoader(testSet, batch_size=32, num_workers=2, shuffle=False)


Upsampling Cardiomegaly...
Upsampling Consolidation...
------------------------------
Found 227395 images in total, 77866 positive images, 149529 negative images
Edema(C1): imbalance ratio is 0.3424
------------------------------
------------------------------
Found 202 images in total, 42 positive images, 160 negative images
Edema(C1): imbalance ratio is 0.2079
------------------------------


In [ ]:
print(len(trainloader))
print(len(testloader))

7107
7


In [ ]:
# paramaters
SEED = 123
BATCH_SIZE = 32
imratio = traindSet.imratio
lr = 0.05 # using smaller learning rate is better
gamma = 500
weight_decay = 1e-5
margin = 1.0

# model
set_all_seeds(SEED)
model = DenseNet121(pretrained=False, last_activation='sigmoid', activations='relu', num_classes=1)
model = model.cuda()

# load pretrained model
if True:
  PATH = 'DenseNet121_pretrained_model.pth' 
  state_dict = torch.load(PATH)
  state_dict.pop('classifier.weight', None)
  state_dict.pop('classifier.bias', None) 
  model.load_state_dict(state_dict, strict=False)


# define loss & optimizer
Loss = AUCMLoss(imratio=imratio)
optimizer = PESG(model, 
                 a=Loss.a, 
                 b=Loss.b, 
                 alpha=Loss.alpha, 
                 imratio=imratio, 
                 lr=lr, 
                 gamma=gamma, 
                 margin=margin, 
                 weight_decay=weight_decay)

In [ ]:
best_val_auc = 0
loss_total = []

for epoch in range(2):
  if epoch > 0:
     optimizer.update_regularizer(decay_factor=10)
  for idx, data in enumerate(trainloader):
      train_data, train_labels = data
      train_data, train_labels = train_data.cuda(), train_labels.cuda()
      y_pred = model(train_data)
      loss = Loss(y_pred, train_labels)
      loss_total += [loss.item()]

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      # validation
      if idx % 400 == 0:
        model.eval()
        with torch.no_grad():    
              test_pred = []
              test_true = [] 
              for jdx, data in enumerate(testloader):
                  test_data, test_label = data
                  test_data = test_data.cuda()
                  y_pred = model(test_data)
                  test_pred.append(y_pred.cpu().detach().numpy())
                  test_true.append(test_label.numpy())
              
              test_true = np.concatenate(test_true)
              test_pred = np.concatenate(test_pred)
              val_auc =  roc_auc_score(test_true, test_pred) 

              writer.add_scalar('Optimizing AUCM Loss/AUCMLoss', np.mean(loss_total), epoch*len(trainloader) + idx)
              writer.add_scalar('Optimizing AUCM Loss/VAL AUC', np.mean(val_auc), epoch*len(trainloader) + idx)

              model.train()

              if best_val_auc < val_auc:
                 best_val_auc = val_auc
                 torch.save(model.state_dict(), 'DenseNet121_OptimizeAUCM_id%d_best_model.pth' % class_id)
              
        print ('Epoch=%s, BatchID=[%s/%s] | AUCM_Loss=%f, Val_AUC=%.4f, lr=%.4f'%(epoch, idx, len(trainloader), loss.item(), val_auc,  optimizer.lr))

print ('Best Val_AUC is %.4f'%best_val_auc)

Epoch=0, BatchID=[0/7107] | AUCM_Loss=0.143530, Val_AUC=0.7485, lr=0.0500
Epoch=0, BatchID=[400/7107] | AUCM_Loss=0.141314, Val_AUC=0.8848, lr=0.0500


KeyboardInterrupt: 

## 3. Multi-Label Training
- Optimizing Multi-Label AUC (5 tasks)


In [ ]:
# dataloader
# Index: -1 denotes multi-label mode including 5 diseases
traindSet = CheXpert(csv_path=root+'train.csv', image_root_path=root, use_upsampling=False, use_frontal=True, image_size=224, mode='train', class_index=-1)
testSet =  CheXpert(csv_path=root+'valid.csv',  image_root_path=root, use_upsampling=False, use_frontal=True, image_size=224, mode='valid', class_index=-1)
trainloader =  torch.utils.data.DataLoader(traindSet, batch_size=32, num_workers=2, shuffle=True)
testloader =  torch.utils.data.DataLoader(testSet, batch_size=32, num_workers=2, shuffle=False)

C:\Users\Hyunmin\Anaconda3\envs\pytorch10\lib\site-packages\libauc\datasets\chexpert.py:35: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df['Path'] = self.df['Path'].str.replace('CheXpert-v1.0-small/', '')
C:\Users\Hyunmin\Anaconda3\envs\pytorch10\lib\site-packages\libauc\datasets\chexpert.py:36: FutureWarning: The default value of regex will change from True to False in a future version.
  self.df['Path'] = self.df['Path'].str.replace('CheXpert-v1.0/', '')


Multi-label mode: True, Number of classes: [5]
------------------------------
Found 191027 images in total, 23385 positive images, 167642 negative images
Cardiomegaly(C0): imbalance ratio is 0.1224

Found 191027 images in total, 61493 positive images, 129534 negative images
Edema(C1): imbalance ratio is 0.3219

Found 191027 images in total, 12983 positive images, 178044 negative images
Consolidation(C2): imbalance ratio is 0.0680

Found 191027 images in total, 59583 positive images, 131444 negative images
Atelectasis(C3): imbalance ratio is 0.3119

Found 191027 images in total, 76899 positive images, 114128 negative images
Pleural Effusion(C4): imbalance ratio is 0.4026

Multi-label mode: True, Number of classes: [5]
------------------------------
Found 202 images in total, 66 positive images, 136 negative images
Cardiomegaly(C0): imbalance ratio is 0.3267

Found 202 images in total, 42 positive images, 160 negative images
Edema(C1): imbalance ratio is 0.2079

Found 202 images in total

In [ ]:
# paramaters
SEED = 123
BATCH_SIZE = 32
 
lr = 0.1 # using smaller learning rate is better
gamma = 500
imratio = traindSet.imratio_list 
weight_decay = 1e-5
margin = 1.0

# model
set_all_seeds(SEED)
model = DenseNet121(pretrained=True, last_activation=None, activations='relu', num_classes=5)
model = model.cuda()

# define loss & optimizer
Loss = AUCM_MultiLabel(imratio=imratio, num_classes=5)
optimizer = PESG(model, 
                 a=Loss.a, 
                 b=Loss.b, 
                 alpha=Loss.alpha, 
                 lr=lr, 
                 gamma=gamma, 
                 margin=margin, 
                 weight_decay=weight_decay, device='cuda')

In [ ]:
# training
best_val_auc = 0 
loss_total = []

for epoch in range(2):
    if epoch > 0:
        optimizer.update_regularizer(decay_factor=10)       
    for idx, data in enumerate(trainloader):
      train_data, train_labels = data
      train_data, train_labels  = train_data.cuda(), train_labels.cuda()
      y_pred = model(train_data)
      y_pred = torch.sigmoid(y_pred)
      loss = Loss(y_pred, train_labels)
      loss_total += [loss.item()]
      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
        
      # validation  
      if idx % 400 == 0:
         model.eval()
         with torch.no_grad():    
              test_pred = []
              test_true = [] 
              for jdx, data in enumerate(testloader):
                  test_data, test_labels = data
                  test_data = test_data.cuda()
                  y_pred = model(test_data)
                  y_pred = torch.sigmoid(y_pred)
                  test_pred.append(y_pred.cpu().detach().numpy())
                  test_true.append(test_labels.numpy())
            
              test_true = np.concatenate(test_true)
              test_pred = np.concatenate(test_pred)
              val_auc_mean =  roc_auc_score(test_true, test_pred) 

              writer.add_scalar('Multi-Label Training/AUCM_MultiLabel', np.mean(loss_total), epoch*len(trainloader) + idx)
              writer.add_scalar('Multi-Label Training/VAL AUC', np.mean(val_auc_mean), epoch*len(trainloader) + idx)
              
              model.train()

              if best_val_auc < val_auc_mean:
                 best_val_auc = val_auc_mean
                 torch.save(model.state_dict(), 'DenseNet121_aucm_multi_label_pretrained_model.pth')

              print ('Epoch=%s, BatchID [%s/%s] | AUCM_Loss=%f, Val_AUC=%.4f, Best_Val_AUC=%.4f'%(epoch, idx, len(trainloader), loss.item(), val_auc_mean, best_val_auc))

Epoch=0, BatchID [0/5970] | AUCM_Loss=0.454441, Val_AUC=0.5777, Best_Val_AUC=0.5777
Epoch=0, BatchID [400/5970] | AUCM_Loss=0.542490, Val_AUC=0.8379, Best_Val_AUC=0.8379


KeyboardInterrupt: 